In [ ]:
# File: belt.ipynb
# Author: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import random

import numpy as np
from frontend import App

app = App.create("belt")

V, F = app.mesh.cylinder(r=0.5, min_x=-0.5, max_x=0.5, n=21)
app.asset.add.tri("cylinder", V, F)

N, Nb, dirs = 3, 3, []
for k in range(Nb):
    r, hw = 1.25 + 2e-3 * k, 0.4 - 0.05 * k
    n = int(44 * hw)
    V, F = app.mesh.cylinder(r=r, min_x=-hw, max_x=hw, n=n)
    app.asset.add.tri(f"belt-{k}", V, F)

for i in range(N):
    r, t = 0.65, 2 * i * np.pi / N
    x, y = r * np.sin(t), r * np.cos(t)
    dirs.append([x, y])

scene = app.scene.create()

for x, y in dirs:
    s, w, axis = 0.511, 60, [0, 0, 1]
    target = [s * x, s * y, 0]
    jitter = 1e-2 * random.random()
    obj = scene.add("cylinder").rotate(90, "y").at(x, y, jitter)
    obj.param.set("friction", 0.5)
    pin = obj.pin()
    pin.move_by(target, 1.0)
    pin.spin(t_start=1.5, t_end=3, angular_velocity=w, axis=axis, center=target)

for k in range(Nb):
    obj = scene.add(f"belt-{k}").rotate(90, "y")
    obj.param.set("strain-limit", 0.05)

opts = {"eyeup": 0.7, "pin": False, "wireframe": True}
scene = scene.build().report()
scene.preview(options=opts)

In [ ]:
session = app.session.create(scene)
(
    session.param.set("dt", 0.01)
    .set("frames", 200)
    .set("gravity", 0.0)
    .set("min-newton-steps", 32)
)
session = session.build()

In [ ]:
session.start().preview(options=opts)
session.stream()

In [ ]:
session.animate(options=opts)

In [ ]:
session.export.animation()

In [ ]:
# this is for CI
assert session.finished()